In [ ]:
from scripts.educ_module import regressiont, knn
import pandas as pd
from sklearn.metrics import mean_squared_error
from math import sqrt
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsRegressor
import numpy as np
from scipy.optimize import minimize
import ipywidgets as widgets
from IPython.display import display
from matplotlib.ticker import FuncFormatter
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# Making Predictions
#### Michelle Wilkerson

The goal of this lesson is to explore different models and methods for making predictions using data related to gray wolves and Yellowstone National Park. Each model comes with its own pros and cons that need to be considered, and we hope that by the end of this exploration, you will be able to think about possible benefits that come with different methodologies and be more critical of prediction techniques that you encounter.

## Background

Yellowstone is a national park located in Wyoming, Montana, and Idaho, covering almost 3,500 square miles. It became the first National Park in the United States in 1872 under President Ulysses S. Grant. It is thought to not only be the first National Park in the United states, but also the first national park in the world.

<img src=http://d1njyp8tsu122i.cloudfront.net/wp-content/uploads/map-lodging-by-state_680x392.jpg width=600>

Yellowstone is famous primarily for two things:

- First, for its geysers and hot springs. It contains about half of the world’s geysers, with over 450 active geysers. For comparison, the next second largest collection of active geysers has about 200. It is home to the world's tallest active geyser, Steamboat Geyser, and to the famous <a href="https://youtu.be/h9uRaEoEV-U?t=1m36s">Old Faithful Geyser</a>. 

- The park is also famous for its wildlife. According to the National Park Service, Yellowstone is "home to the largest concentration of mammals in the lower 48 states," with 67 species of mammals, and over 325 different species of birds, fish, amphibians, and reptiles.

Despite being a National Park, the animals of Yellowstone were not originally offered special protections.
The <a href=https://www.nps.gov/yell/learn/management/yellowstoneprotectionact1872.htm>Yellowstone National Park Protection Act</a> said that the Secretary of the Interior would "provide against the wanton destruction of the fish and game found within [Yellowstone]," but made no specific provisions, and left wiggle room for interpretation. Of semi-recent attention has been the status of gray wolves in the park -- which are neither game nor fish, so under the Protection Act, were left on their own. 

<img src=http://d1njyp8tsu122i.cloudfront.net/wp-content/uploads/Yellowstone-Wildlife-Wolf-11.jpg width=400>

Historically, gray wolves roamed about <a href="https://www.nwf.org/Wildlife/Wildlife-Library/Mammals/Gray-Wolf.aspx">two-thirds</a> of the United States, with numbers estimated to be over a quarter-million, but by the 1960s, the only wild gray wolves in the lower 48 states were restricted to <a href="http://www.missionwolf.org/page/wild-wolf-history/">Michigan and Minnesota</a>.

With America's westward expansion came the tolling of the bells for wolves. Not only did they have to compete with settlers for space, but they were also hunted. Farmers and ranchers feared for their livestock, and successfully advocated for programs to reduce gray wolf populations. Beginning in the 19th century and continuing into the 20th, there were <a href="https://www.fws.gov/midwest/wolf/aboutwolves/biologue.htm">government programs that offered bounties for each wolf killed</a>. In 1880, the superintendent of Yellowstone had stated that <a href="http://www.pbs.org/wnet/nature/the-wolf-that-changed-america-wolf-wars-americas-campaign-to-eradicate-the-wolf/4312/">"the value of their [wolves and coyotes] hides and their easy slaughter with strychnine-poisoned carcasses have nearly led to their extermination.”</a> By the end of the 1920s, gray wolves had disappeared from most of the United States. 

<img src=http://www.hcn.org/issues/46.21/have-returning-wolves-really-saved-yellowstone/yellowstonewolves1-jpg/@@images/7fdf9c1c-9425-45dd-99a7-fe6f43ffa3a1.jpeg width=400>

In 1973, Greater Yellowstone was named as one of three recovery areas for the endangered gray wolf. From 1995 to 1997, 41 wild wolves from Montana and Canada were released in Yellowstone National Park, where their populations have been climbing since. 

### Wolves in Yellowstone
Wolves in Yellowstone primarily feed on hooved animals, the most common being elk, and deer and smaller animals during the summer. When the wolves were reintroduced to the park, of course we would expect some sort of effect on the populations of those animals of prey. In this notebook, we are going to investigate what those effects were on the elk populations, and also on the wolf populations.

---

## Navigation

Before we begin analyzing our data, we need to run some cells of code. You don't need to know what these cells are doing in any detail, for now. To run a cell of code, click on the cell, then type `SHIFT-ENTER` or simply click the button on the toolbar at the top of the page that looks like ▶| . Run each cell of code as you work down the notebook, in the order they are placed. Some cells rely on others above it, so if you don't run them in order, you may encounter some puzzling errors.

---

## Reading in data

Our first data set contains the wolf and elk populations from 1994 to 2012, though there are some years missing in the late 90s. Run the cell below to view a print-out of the table we will be working with.

In [ ]:
# reading in our data from an existing file
data = pd.read_csv('data/wolf_and_elk_in_yellowstone.csv', thousands=',').drop('Notes', axis=1).dropna().reset_index(drop=True)
data

To get a sense of the data, it is helpful to visualize it with graphs. The cell below constructs two graphs from the data table above and plots the change in each animal's population numbers over time.

In [ ]:
f, axs = plt.subplots(1,2,figsize=(15, 6))

axs[0].plot(data['Year'], data['Elk Population'])
axs[0].set_title('Elk')

axs[1].plot(data['Year'], data['Wolf Population'])
axs[1].set_title('Wolves')

Plotting the populations separately makes it difficult to see the the absolute changes in population relative to one another, so next, we will plot them on the same y-axis.

In [ ]:
data.plot('Year', figsize=(9,6))

Since the scale of the absolute numbers for both animals are so different, another helpful view may be to look at the percent change in population from year to year. First we have to write a function that will set up our visualization:

In [ ]:
def percent_changing(periods=1):
    percent_changed = data.pct_change(periods)
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    
    plt.plot(data['Year'], percent_changed['Elk Population'], c='b')
    plt.plot(data['Year'], percent_changed['Wolf Population'], c='g')
    plt.axhline(y=0, c="r")
    plt.legend()
    ax.yaxis.set_major_formatter(FuncFormatter('{0:.0%}'.format))
    plt.show()

Play with the slider to increase the length of the period (the number of years) that the percent change is measured over.

In [ ]:
slider = widgets.IntSlider(min=1,max=6,step=1,value=1)
display(widgets.interactive(percent_changing, periods=slider))

<font color="blue">Question:</font> Based on the data and the graphs above, what do you think is going on with the wolf and elk populations? How do you think they will change going forward? Are they close to equilibrium?

*Write your response in the cell below, then type*  `SHIFT-ENTER`

<font color="blue">Question: </font> How do you think the changing populations of wolves and elk will impact Yellowstone park and its other animal populations? Specifically, how do you think these trends will affect the populations of wolves' other prey? What about other predatory species that also hunt those prey? How about flora populations?

*Write your response in the cell below, then type*  `SHIFT-ENTER`

## Prediction

Prediction is a powerful tool that we can use to make guesses about how data will look in the future based on previous trends. We are going to start discussing some basic prediction techniques, then move on to some more complex techniques to predict how the populations of the animals we've been talking about might look in the future.

#### Single Point Model

First, the Single Point model. This method of prediction takes one data point from our existing data and uses it to predict the populations for the next year. 

<font color='blue'>Question:</font> What issues might arise from using this method?

### Linear Regression Model for Predicting Population

Linear regression can be done in two ways: First, we can create a regression line using all of the data points that we currently have to make a prediction about the population in future years. We could also use $N$-most recent points from our data set to make a prediction about future years' populations, rather than using all of our data.

<font color='blue'>Question:</font> Think about the differences between these two models. How are they similar? How are the findings/conclusions similar or different? What are the pros and cons of each?

In the cell below, we have some code that implements a linear regression model on our data. We will use this to investigate how changing the number of data points that we consider when doing regression influences the prediction of the next year's population. 

**Wolves:**

Run the cell below and you'll see a visualization with a slider bar that allows you to select the number of data points to consider for the linear regression model. Try to take note of how the prediction for the population number is affected when you consider more vs. fewer data points in the regression model.  

In [ ]:
p_slider = widgets.IntSlider(min=1, max=16, step=1, value=1)
display(widgets.interactive(regressiont, included_points=p_slider, dependent="Wolf Population"))

We can estimate the slop of this line and the intercept, estimating the regression model:

$WOLFPOP_i= \alpha + \beta YEAR + \epsilon_i$

Let's look at the summary of the model:

In [ ]:
mod = smf.ols(formula='Wolf_Pop ~ Year', data=data.rename(columns={"Wolf Population": "Wolf_Pop"}))
res = mod.fit()
print(res.summary())

It looks like our model is quite good, and the Wolf population does indeed grow at a near linear pace. But knowing what we do about ecology, we would think this would level off soon as the Elk population also decreases.

Can we guess the wolf population in 10 years (from 2013) with this model?

In [ ]:
res.predict({"Year": [2013+10]})

That's quite high! Just how accurate is our model? The box description above tells us in many different ways. Perhaps an easier conceptual evaluation is the **RMSE**, or Root Mean Squared Error. This metric is exactly what it means, it takes the square root of the average squared error on each observation:

$\operatorname{RMSE}=\sqrt{\frac{\sum_{t=1}^n (\hat y_t - y_t)^2}{n}}$

In [ ]:
print("RMSE: " + str(sqrt(mean_squared_error(data['Wolf Population'], res.predict({"Year": data['Year']})))))

Not bad! But again, we can't expect a population to grow linearly.

---

**Elk**

We can do the same for Elk:

In [ ]:
p_slider = widgets.IntSlider(min=1, max=16, step=1, value=1)
display(widgets.interactive(regressiont,included_points=p_slider, dependent="Elk Population"))

Likewise:

$ELKPOP_i= \alpha + \beta YEAR + \epsilon_i$

In [ ]:
mod = smf.ols(formula='Elk_Pop ~ Year', data=data.rename(columns={"Elk Population": "Elk_Pop"}))
res = mod.fit()
print(res.summary())

In 10 years:

In [ ]:
res.predict({"Year": [2013+10]})

***Oh no!*** That's not possible. How's our model for Elk?

**RMSE**

In [ ]:
print("RMSE: " + str(sqrt(mean_squared_error(data['Elk Population'], res.predict({"Year": data['Year']})))))

Much worse.

---

So when do the linear models think the Elk will die out?

In [ ]:
# elk
mod = smf.ols(formula='Elk_Pop ~ Year', data=data.rename(columns={"Elk Population": "Elk_Pop"}))
model_elk = mod.fit()

# wolves
mod = smf.ols(formula='Wolf_Pop ~ Year', data=data.rename(columns={"Wolf Population": "Wolf_Pop"}))
model_wolf = mod.fit()

time_period = list(data['Year']) + list(range(2014,2030))
X = time_period

plt.scatter(time_period, model_wolf.predict({"Year": X}))
plt.plot(time_period, model_wolf.predict({"Year": X}))
plt.scatter(time_period, model_elk.predict({"Year": X}))
plt.plot(time_period, model_elk.predict({"Year": X}))
plt.title("Elk vs. Wolf Population, Linear Model")
plt.show()

As we've noted above, it looks like the couple years before 2013 the trend is changing (although this curves happens a couple times), but this is what a simple linear regression would predict.

<font color='blue'>Question:</font> Given your qualitative understanding of this phenomenon, what can you say about a linear model's prediction?

---

### K-Nearest Neighbor Model for Predicting Population

We know better than to trust linear models for population modeling. Let's jump to a simple, yet more sophisticated modeling technique. K-Nearest Neighbors (a.k.a. kNN) is an algorithm used for prediction that compares features of a new data point to the $K$ existing data points that are closest to it ("close"ness depending on the measures of similarity you decide to use) to make a prediction about that data point.

A basic example of kNN regression is using the height of neighboring buildings to predict the height of a new one. In this example, if we're predicting the height of a new building in downtown Manhattan, it doesn't matter what the height of buildings are in Cedar Rapids, Iowa, it's more helpful to concentrate on the height of other skyscrapers in Manhattan. We might want to predict the number of stories that a new Manhattan apartment complex will have based on the apartment complexes that are around it.

In a similar line of thinking, just as we were selecting points to include for our linear regression, we might want to have our model only use the most relevant data points when predicting a new one.

Let's see what this would look like on dummy data:

In [ ]:
from scripts.educ_module import knn

slider = widgets.IntSlider(min=1,max=6,step=1,value=1)
display(widgets.interactive(knn, neighbors=slider))

Now we can use another slider to change the number of closest observations the model will use in predicting the next observation.

In [ ]:
slider = widgets.IntSlider(min=1,max=6,step=1,value=1)
display(widgets.interactive(knn, neighbors=slider))

We can use the same concept to model our wolf and elk population differently than the linear regression above:

In [ ]:
def knn_yellowstone(neighbors=1, dependent="Wolf Population"):
    model = KNeighborsRegressor(neighbors)
    X = [[x] for x in data['Year']]
    y_values = data[dependent]

    model.fit(X, y_values)
    model.predict(X)

    plt.scatter(data['Year'],model.predict(X))
    plt.plot(data['Year'],model.predict(X))
    plt.scatter(data['Year'],data[dependent])
    plt.plot(data['Year'],data[dependent])
    plt.show()

    print("RMSE: " + str(sqrt(mean_squared_error(data[dependent], model.predict(X)))))

In [ ]:
slider = widgets.IntSlider(min=1,max=16,step=1,value=5)
display(widgets.interactive(knn_yellowstone, neighbors=slider, dependent='Wolf Population'))

We can also predict 10 years out like above:

In [ ]:
def ten_years_out_wolves(neighbors):
    model_wolves = KNeighborsRegressor(neighbors)
    X = [[x] for x in data['Year']]
    y_values = data['Wolf Population']
    model_wolves.fit(X, y_values)
    print("Wolves in 10 years: " + str(model_wolves.predict([[2013 + 10]])))

In [ ]:
slider = widgets.IntSlider(min=1,max=16,step=1,value=5)
display(widgets.interactive(ten_years_out_wolves, neighbors=slider, dependent='Wolf Population'))

---

Let's do the same for Elk:

In [ ]:
slider = widgets.IntSlider(min=1,max=16,step=1,value=5)
display(widgets.interactive(knn_yellowstone, neighbors=slider, dependent='Elk Population'))

In [ ]:
def ten_years_out_elk(neighbors):
    model_wolves = KNeighborsRegressor(neighbors)
    X = [[x] for x in data['Year']]
    y_values = data['Elk Population']
    model_wolves.fit(X, y_values)
    print("Elk in 10 years: " + str(model_wolves.predict([[2013 + 10]])))

In [ ]:
slider = widgets.IntSlider(min=1,max=16,step=1,value=5)
display(widgets.interactive(ten_years_out_elk, neighbors=slider, dependent='Elk Population'))

---

Those numbers look much more reasonable. Let's put this all together to model the future with out new technique, KNN:

In [ ]:
def future_population_knn(neighbors):

    # wolves
    model_wolves = KNeighborsRegressor(neighbors)
    X = [[x] for x in data['Year']]
    y_values = data['Wolf Population']
    model_wolves.fit(X, y_values)
    model_wolves.predict([[2013 + 10]])

    # elk
    model_elk = KNeighborsRegressor(neighbors)
    X = [[x] for x in data['Year']]
    y_values = data['Elk Population']
    model_elk.fit(X, y_values)
    
    time_period = list(data['Year']) + list(range(2014,2030))
    X = [[x] for x in time_period]
    
    plt.scatter(time_period,model_wolves.predict(X))
    plt.plot(time_period,model_wolves.predict(X))
    plt.scatter(time_period,model_elk.predict(X))
    plt.plot(time_period,model_elk.predict(X))
    plt.title("Elk vs. Wolf Population")
    plt.show()

In [ ]:
slider = widgets.IntSlider(min=1,max=16,step=1,value=5)
display(widgets.interactive(future_population_knn, neighbors=slider))

This looks more reasonable, but the straight lines at the end are not practical for a life cycle. Nevertheless, we will always have updated population numbers each year that we can add to this, so that the next few years may actually be quite accurate!

---

Prey is a very important factor in predator-prey interaction with a profound effect on the wolf population. Predators kill and eat other organisms, their prey. If or when a predator runs out of prey, they run the risk of dying out. Usually, the populations of predators and prey in an ecosystem oscillate in a cycle, which, surprisingly, can often be described using mathematical models. In this cycle, the prey population typically peaks just before the predator population does (usually at about ¼ of a cycle). 

Take a moment to think about what other factors might be related to wolf populations. There are definitely a lot of factors! 


* What other factors might be related to wolf populations?

    
* Predator-Prey interactions
    * Prey is a very important factor, one of many factors
    * How do predators and their prey affect one another?
        * Predators kill and eat other organisms, their prey
        * Predator runs out of prey, dies out
    * Prey population typically peaks before predator population (usually ¼ of a cycle), natural oscillation
    
    
* Explain K-nearest neighbors (in layman terms)
    

* We can only choose some factors - which should we take into account? Why?


* The factors we chose/will use:
    * Elk population?
    * Bison population?
        * potential source for number of bison: http://ibmp.info/Library/OpsPlans/2016_StatusYellowstoneBisonPopulation_Sep2016.pdf
            * note on usage: population control measures are commonly used in yellowstone for bison